#### Installs

In [ ]:
!pip install numpy matplotlib scipy sklearn hmmlearn simplejson eyed3 pydub
!pip install pyAudioAnalysis
!pip install plot_metrics

ERROR: Could not find a version that satisfies the requirement plot_metrics (from versions: none)
ERROR: No matching distribution found for plot_metrics


#### Imports

In [ ]:
import fnmatch
import os
import zipfile

from pyAudioAnalysis import audioBasicIO 
from pyAudioAnalysis import audioSegmentation as aS
import scipy.io.wavfile as wavfile
import wave

import numpy as np
from numpy.lib import stride_tricks
import os
from PIL import Image
import scipy.io.wavfile as wav

import pandas as pd

import random
import boto

from sklearn.metrics import confusion_matrix, roc_curve, auc
from keras.models import Sequential
from keras.layers import Dense, Dropout, Activation, Flatten
from keras.layers import Conv2D, MaxPooling2D
from keras.utils import np_utils
from keras import backend as K

Using TensorFlow backend.


#### Download DIAC dataset

Information about dataset struture could be found here: https://dcapswoz.ict.usc.edu/wwwutil_files/DAICWOZDepression_Documentation.pdf

In [ ]:
!mkdir DAIC-WOZ
%cd DAIC-WOZ

/content/DAIC-WOZ


In [ ]:
!wget -r -np -nH --cut-dirs=3 -R index.html --user=daicwozuser --ask-password --no-check-certificate https://dcapswoz.ict.usc.edu/wwwdaicwoz/

Password for user ‘daicwozuser’: 
--2020-07-08 23:34:38--  https://dcapswoz.ict.usc.edu/wwwdaicwoz/
Resolving dcapswoz.ict.usc.edu (dcapswoz.ict.usc.edu)... 128.125.133.76
Connecting to dcapswoz.ict.usc.edu (dcapswoz.ict.usc.edu)|128.125.133.76|:443... connected.
  Unable to locally verify the issuer's authority.
HTTP request sent, awaiting response... 401 Unauthorized
Authentication selected: Basic realm="Restricted Content"
Reusing existing connection to dcapswoz.ict.usc.edu:443.
HTTP request sent, awaiting response... 200 OK
Length: unspecified [text/html]
Saving to: ‘index.html.tmp’

index.html.tmp          [ <=>                ]  39.62K  --.-KB/s    in 0.04s   

2020-07-08 23:34:38 (1.10 MB/s) - ‘index.html.tmp’ saved [40568]

Loading robots.txt; please ignore errors.
--2020-07-08 23:34:38--  https://dcapswoz.ict.usc.edu/robots.txt
Reusing existing connection to dcapswoz.ict.usc.edu:443.
HTTP request sent, awaiting response... 404 Not Found
2020-07-08 23:34:38 ERROR 404: Not Found

In [ ]:
%cd ..

/content


#### Data preparation 

Extract sound files from zip sessions

In [ ]:
def extract_files(zip_file, out_dir, delete_zip=False):
    """
    A function takes in a zip file and extracts the .wav file and
    *TRANSCRIPT.csv files into separate folders in a user
    specified directory.

    Parameters
    ----------
    zip_file : filepath
        path to the folder containing the DAIC-WOZ zip files
    out_dir : filepath
        path to the desired directory where audio and transcript folders
        will be created
    delete_zip : bool
        If true, deletes the zip file once relevant files are extracted

    Returns
    -------
    Two directories :
        audio : containing the extracted wav files
        transcripts : containing the extracted transcript csv files
    """
    # create audio directory
    audio_dir = os.path.join(out_dir, 'audio')
    if not os.path.exists(audio_dir):
        os.makedirs(audio_dir)

    # create transcripts directory
    transcripts_dir = os.path.join(out_dir, 'transcripts')
    if not os.path.exists(audio_dir):
        os.makedirs(transcripts_dir)

    zip_ref = zipfile.ZipFile(zip_file)
    for f in zip_ref.namelist():  # iterate through files in zip file
        if f.endswith('.wav'):
            zip_ref.extract(f, audio_dir)
        elif fnmatch.fnmatch(f, '*TRANSCRIPT.csv'):
            zip_ref.extract(f, transcripts_dir)
    zip_ref.close()

    if delete_zip:
        os.remove(zip_file)

In [ ]:
dir_name = '/content/DAIC-WOZ'

# directory where audio and transcripts folders will be created
out_dir = '/depression-detection/data/raw'

# delete zip file after file wav and csv extraction
delete_zip = True

# iterate through zip files in dir_name and extracts wav and transcripts
for file in os.listdir(dir_name):
    if file.endswith('.zip'):
      zip_file = os.path.join(dir_name, file)
      extract_files(zip_file, out_dir, delete_zip=delete_zip)

#### Data Segmentation

In [ ]:
def remove_silence(filename, out_dir, smoothing=1.0, weight=0.3, plot=False):
    """
    A function that implements pyAudioAnalysis' silence extraction module
    and creates wav files of the participant specific portions of audio. The
    smoothing and weight parameters were tuned for the AVEC 2016 dataset.

    Parameters
    ----------
    filename : filepath
        path to the input wav file
    out_dir : filepath
        path to the desired directory (where a participant folder will
        be created containing a 'PXXX_no_silence.wav' file)
    smoothing : float
        tunable parameter to compensate for sparseness of recordings
    weight : float
        probability threshold for silence removal used in SVM
    plot : bool
        plots SVM probabilities of silence (used in tuning)

    Returns
    -------
    A folder for each participant containing a single wav file
    (named 'PXXX_no_silence.wav') with the vast majority of silence
    and virtual interviewer speech removed. Feature extraction is
    performed on these segmented wav files.
    """
    partic_id = 'P' + filename.split('/')[-1].split('_')[0]  # PXXX
    if is_segmentable(partic_id):
        # create participant directory for segmented wav files
        participant_dir = os.path.join(out_dir, partic_id)
        if not os.path.exists(participant_dir):
            os.makedirs(participant_dir)

        os.chdir(participant_dir)

        [Fs, x] = audioBasicIO.read_audio_file(filename)
        segments = aS.silence_removal(x, Fs, 0.020, 0.020,
                                     smooth_window=smoothing,
                                     weight=weight,
                                     plot=plot)

        for s in segments:
            seg_name = "{:s}_{:.2f}-{:.2f}.wav".format(partic_id, s[0], s[1])
            wavfile.write(seg_name, Fs, x[int(Fs * s[0]):int(Fs * s[1])])

        # concatenate segmented wave files within participant directory
        concatenate_segments(participant_dir, partic_id)


def is_segmentable(partic_id):
    """
    A function that returns True if the participant's interview clip is not
    in the manually identified set of troubled clips. The clips below were
    not segmentable do to excessive static, proximity to the virtual
    interviewer, volume levels, etc.
    """
    troubled = set(['P300', 'P305', 'P306', 'P308', 'P315', 'P316', 'P343',
                    'P354', 'P362', 'P375', 'P378', 'P381', 'P382', 'P385',
                    'P387', 'P388', 'P390', 'P392', 'P393', 'P395', 'P408',
                    'P413', 'P421', 'P438', 'P473', 'P476', 'P479', 'P490',
                    'P492'])
    return partic_id not in troubled


def concatenate_segments(participant_dir, partic_id, remove_segment=True):
    """
    A function that concatenates all the wave files in a participants
    directory in to single wav file (with silence and other speakers removed)
    and writes in to the participant's directory, then removes the individual
    segments (when remove_segment=True).
    """
    infiles = os.listdir(participant_dir)  # list of wav files in directory
    outfile = '{}_no_silence.wav'.format(partic_id)

    data = []
    for infile in infiles:
        w = wave.open(infile, 'rb')
        data.append([w.getparams(), w.readframes(w.getnframes())])
        w.close()
        if remove_segment:
            os.remove(infile)

    output = wave.open(outfile, 'wb')
    # details of the files must be the same (channel, frame rates, etc.)
    output.setparams(data[0][0])

    # write each segment to output
    for idx in range(len(data)):
        output.writeframes(data[idx][1])
    output.close()

In [ ]:
dir_name = '/content/depression-detection/raw/audio'

# directory where a participant folder will be created containing their
# segmented wav file
out_dir = '/content/depression-detection/interim'

# iterate through wav files in dir_name and create a segmented wav_file
for file in os.listdir(dir_name):
    if file.endswith('.wav'):
        filename = os.path.join(dir_name, file)
        remove_silence(filename, out_dir)

#### Spectrograms

In [ ]:
def stft(sig, frameSize, overlapFac=0.5, window=np.hanning):
    """
    Short-time Fourier transform of audio signal.
    """
    win = window(frameSize)
    hopSize = int(frameSize - np.floor(overlapFac * frameSize))
    # zeros at beginning (thus center of 1st window should be for sample nr. 0)
    frame = np.floor((frameSize)/2.0)
    frame = frame.astype(np.int64)
    size = np.zeros(frame)
    samples = np.append((size), sig)
    # cols for windowing
    cols = int(np.ceil( (len(samples) - frameSize) / float(hopSize)) + 1)
    # zeros at end (thus samples can be fully covered by frames)
    samples = np.append(samples, np.zeros(frameSize))

    frames = stride_tricks.as_strided(samples, shape=(cols, frameSize),
                                      strides=(samples.strides[0]*hopSize,
                                      samples.strides[0])).copy()
    frames *= win

    return np.fft.rfft(frames)


def logscale_spec(spec, sr=44100, factor=20.):
    """
    Scale frequency axis logarithmically.
    """
    timebins, freqbins = np.shape(spec)

    scale = np.linspace(0, 1, freqbins) ** factor
    scale *= (freqbins-1)/max(scale)
    scale = np.unique(np.round(scale))

    # create spectrogram with new freq bins
    newspec = np.complex128(np.zeros([timebins, len(scale)]))
    for i in range(0, len(scale)):
        if i == len(scale)-1:
            newspec[:, i] = np.sum(spec[:, scale[i]:], axis=1)
        else:
            newspec[:, i] = np.sum(spec[:, scale[i]:scale[i+1]], axis=1)

    # list center freq of bins
    allfreqs = np.abs(np.fft.fftfreq(freqbins*2, 1./sr)[:freqbins+1])
    freqs = []
    for i in range(0, len(scale)):
        if i == len(scale)-1:
            freqs += [np.mean(allfreqs[scale[i]:])]
        else:
            freqs += [np.mean(allfreqs[scale[i]:scale[i+1]])]

    return newspec, freqs


def stft_matrix(audiopath, binsize=2**10, png_name='tmp.png',
                save_png=False, offset=0):
    """
    A function that converts a wav file into a spectrogram represented by a \
    matrix where rows represent frequency bins, columns represent time, and \
    the values of the matrix represent the decibel intensity. A matrix of \
    this form can be passed as input to the CNN after undergoing normalization.
    """
    samplerate, samples = wav.read(audiopath)
    s = stft(samples, binsize)

    sshow, freq = logscale_spec(s, factor=1, sr=samplerate)
    ims = 20.*np.log10(np.abs(sshow)/10e-6)  # amplitude to decibel
    timebins, freqbins = np.shape(ims)

    ims = np.transpose(ims)
    ims = np.flipud(ims)  

    if save_png:
        create_png(ims, png_name)

    return ims


def create_png(im_matrix, png_name):
    """
    Save grayscale png of spectrogram.
    """
    image = Image.fromarray(im_matrix)
    image = image.convert('L')  # convert to grayscale
    image.save(png_name)

In [ ]:
 # directory containing participant folders with segmented wav files
dir_name = '/content/depression-detection/interim'

# walks through wav files in dir_name and creates pngs of the spectrograms.
# This is a visual representation of what is passed to the CNN before
# normalization, although a cropped matrix representation is actually
# passed.
for subdir, dirs, files in os.walk(dir_name):
    for file in files:
        if file.endswith('.wav'):
          wav_file = os.path.join(subdir, file)
          png_name = subdir + '/' + file[:-4] + '.png'
          print('Processing ' + file + '...')
          stft_matrix(wav_file, png_name=png_name, save_png=True)

#### Train/Test split

In [ ]:
df_train = pd.read_csv('/content/depression-detection/raw/labels/train_split_Depression_AVEC2017.csv')

df_test = pd.read_csv('/content/depression-detection/raw/labels/test_split_Depression_AVEC2017.csv')

df_dev = pd.concat([df_train, df_test], axis=0)


#### Build Spectrograms Dictionaries

In [ ]:
def build_class_dictionaries(dir_name):
    """
    Builds a dictionary of depressed participants and non-depressed
    participants with the participant id as the key and the matrix
    representation of the no_silence wav file as the value. These
    values of this dictionary are then randomly cropped and sampled
    from to create balanced class and speaker inputs to the CNN.
    Parameters
    ----------
    dir_name : filepath
        directory containing participant's folders (which contains the
        no_silence.wav)
    Returns
    -------
    depressed_dict : dictionary
        dictionary of depressed individuals with keys of participant id
        and values of with the matrix spectrogram representation
    normal_dict : dictionary
        dictionary of non-depressed individuals with keys of participant id
        and values of with the matrix spectrogram representation
    """
    depressed_dict = dict()
    normal_dict = dict()
    for subdir, dirs, files in os.walk(dir_name):
        for file in files:
            if file.endswith('no_silence.wav'):
                partic_id = int(file.split('_')[0][1:])
                if in_dev_split(partic_id):
                    wav_file = os.path.join(subdir, file)
                    # matrix representation of spectrogram
                    mat = stft_matrix(wav_file)
                    depressed = get_depression_label(partic_id)  # 1 if True
                    if depressed:
                        depressed_dict[partic_id] = mat
                    elif not depressed:
                        normal_dict[partic_id] = mat
    return depressed_dict, normal_dict

In [ ]:
def in_dev_split(partic_id):
    """
    Returns True if the participant is in the AVEC development split
    (aka participant's we have depression labels for)
    """
    return partic_id in set(df_dev['Participant_ID'].values)

def get_depression_label(partic_id):
    """
    Returns participant's PHQ8 Binary label. 1 representing depression;
    0 representing no depression.
    """
    return df_dev.loc[df_dev['Participant_ID'] ==
                      partic_id]['PHQ8_Binary'].item()


In [ ]:
dir_name = '/content/depression-detection/interim'
depressed_dict, normal_dict = build_class_dictionaries(dir_name)

#### Random Sampling

In [ ]:
np.random.seed(15)  # for reproducibility
access_key = os.environ['AWS_ACCESS_KEY']
access_secret_key = os.environ['AWS_SECRET_KEY']


"""
There exists a large data imbalance between positive and negative samples,
which incurs a large bias in classification. The number of non-depressed
subjects is about four times bigger than that of depressed ones. If these
samples for learning, the model will have a strong bias to the non-depressed
class.
To solve the problem, we perform random cropping on each of the participant's
spectrograms of a specified width (time) and constant height (frequency), to
ensure the CNN has an equal proportion for every subject and each class.
"""


def determine_num_crops(depressed_dict, normal_dict, crop_width=125):
    """
    Finds the shortest clip in the entire dataset which, according to our
    random sampling strategy, will limit the number of samples we take from
    each clip to make sure our classes are balanced.

    Parameters
    ----------
    depressed_dict : dictionary
        a dictionary of depressed participants with the participant id as the
        key and the segmented and concatenated matrix representation of
        their spectrograms as the values.
    crop_width : integer
        the desired pixel width of the crop samples
        (125 pixels = 4 seconds of audio)

    Returns
    -------
    num_samples_from_clips : int
        the maximum number of samples that should be sampled from each clip
        to ensure balanced classes can be built.
    """
    merged_dict = dict(normal_dict, **depressed_dict)
    shortest_clip = min(merged_dict.items(), key=lambda x: x[1].shape[1])
    shortest_pixel_width = shortest_clip[1].shape[1]
    num_samples_from_clips = shortest_pixel_width / crop_width
    return num_samples_from_clips


def build_class_sample_dict(segmented_audio_dict, n_samples, crop_width):
    """
    Get N (num_samples) pseudo random non-overlapping samples from the all
    the depressed participants.

    Parameters
    ----------
    segmented_audio_dict : dictionary
        a dictionary of a class of participants with keys of participant ids
        and values of the segmented audio matrix spectrogram representation
    n_samples : integer
        number of random non-overlapping samples to extract from each
        segmented audio matrix spectrogram
    crop_width : integer
        the desired pixel width of the crop samples
        (125 pixels = 4 seconds of audio)

    Returns
    -------
    class sample dict : dictionary
        a dictionary of a class of participants with keys of participant ids
        and values of a list of the cropped samples from the spectrogram
        matrices.
    """
    class_samples_dict = dict()
    for partic_id, clip_mat in segmented_audio_dict.iteritems():
            samples = get_random_samples(clip_mat, n_samples, crop_width)
            class_samples_dict[partic_id] = samples
    return class_samples_dict


def get_random_samples(matrix, n_samples, crop_width):
    """
    Get N random samples with width of crop_width from the numpy matrix
    representing the participant's audio spectrogram.
    """
    # crop full spectrogram into segments of width = crop_width
    clipped_mat = matrix[:, (matrix.shape[1] % crop_width):]
    n_splits = clipped_mat.shape[1] / crop_width
    cropped_sample_ls = np.split(clipped_mat, n_splits, axis=1)

    # get random samples
    samples = random.sample(cropped_sample_ls, n_samples)
    return samples


def create_sample_dicts(crop_width):
    """
    Utilizes the above function to return two dictionaries, depressed
    and normal. Each dictionary has only participants in the specific class,
    with participant ids as key, a values of a list of the cropped samples
    from the spectrogram matrices. The lists are vary in length depending
    on the length of the interview clip. The entries within the list are
    numpy arrays with dimennsion (513, 125).
    """
    # build dictionaries of participants and segmented audio matrix
    depressed_dict, normal_dict = build_class_dictionaries('/content/depression-detection/interim')
    n_samples = determine_num_crops(depressed_dict, normal_dict,
                                    crop_width=crop_width)
    # get n_sample random samples from each depressed participant
    depressed_samples = build_class_sample_dict(depressed_dict, n_samples,
                                                crop_width)
    # get n_sample random samples from each non-depressed participant
    normal_samples = build_class_sample_dict(normal_dict, n_samples,
                                             crop_width)
    # iterate through samples dictionaries and save a npz file
    # with the radomly sleected n_samples for each participant.
    # save depressed arrays to .npz
    for key, _ in depressed_samples.iteritems():
        path = '/content/depression-detection/processed/'
        filename = 'D{}.npz'.format(key)
        outfile = path + filename
        np.savez(outfile, *depressed_samples[key])
    # save normal arrays to .npz
    for key, _ in normal_samples.iteritems():
        path = '/content/depression-detection/processed'
        filename = '/N{}.npz'.format(key)
        outfile = path + filename
        np.savez(outfile, *normal_samples[key])


def rand_samp_train_test_split(npz_file_dir):
    """
    Given the cropped segments from each class and particpant, this fucntion
    determines how many samples we can draw from each particpant and how many
    participants we can draw from each class.

    Parameters
    ----------
    npz_file_dir : directory
        directory contain the
    crop_width : integer
        the desired pixel width of the crop samples
        (125 pixels = 4 seconds of audio)

    Returns
    -------
    num_samples_from_clips : int
        the maximum number of samples that should be sampled from each clip
        to ensure balanced classes can be built.
    """
    # files in directory
    npz_files = os.listdir(npz_file_dir)

    dep_samps = [f for f in npz_files if f.startswith('D')]
    norm_samps = [f for f in npz_files if f.startswith('N')]
    # calculate how many samples to balance classes
    max_samples = min(len(dep_samps), len(norm_samps))

    # randomly select max participants from each class without replacement
    dep_select_samps = np.random.choice(dep_samps, size=max_samples,
                                        replace=False)
    norm_select_samps = np.random.choice(norm_samps, size=max_samples,
                                         replace=False)

    # randomly select n_samples_per_person (40 in the case of a crop width
    # of 125) from each of the participant lists

    # REFACTOR this code!
    test_size = 0.2
    num_test_samples = int(len(dep_select_samps) * test_size)

    train_samples = []
    for sample in dep_select_samps[:-num_test_samples]:
        npz_file = npz_file_dir + '/' + sample
        with np.load(npz_file) as data:
            for key in data.keys():
                train_samples.append(data[key])
    for sample in norm_select_samps[:-num_test_samples]:
        npz_file = npz_file_dir + '/' + sample
        with np.load(npz_file) as data:
            for key in data.keys():
                train_samples.append(data[key])
    train_labels = np.concatenate((np.ones(len(train_samples)/2),
                                   np.zeros(len(train_samples)/2)))

    test_samples = []
    for sample in dep_select_samps[-num_test_samples:]:
        npz_file = npz_file_dir + '/' + sample
        with np.load(npz_file) as data:
            for key in data.keys():
                test_samples.append(data[key])
    for sample in norm_select_samps[-num_test_samples:]:
        npz_file = npz_file_dir + '/' + sample
        with np.load(npz_file) as data:
            for key in data.keys():
                test_samples.append(data[key])
    test_labels = np.concatenate((np.ones(len(test_samples)/2),
                                  np.zeros(len(test_samples)/2)))

    return np.array(train_samples), train_labels, np.array(test_samples), \
        test_labels


def save_to_bucket(file, obj_name):
    """
    Saves local file to S3 bucket for redundancy and reproducibility.
    """
    conn = boto.connect_s3(access_key, access_secret_key)
    bucket = conn.get_bucket('deprission-det')
    file_object = bucket.new_key(obj_name)
    file_object.set_contents_from_filename(file)

In [ ]:
# build participant's cropped npz files
# this is of the whole no_silence particpant's no_silence interview,
# but each array in the npz files was width of crop_width
create_sample_dicts(crop_width=125)

# random sample from particpants npz files to ensure class balance
train_samples, train_labels, test_samples, \
    test_labels = rand_samp_train_test_split('/content/depression-detection/processed')

# save as npz locally
print("Saving npz file locally...")
np.savez('/content/depression-detection/processed/train_samples.npz', train_samples)
np.savez('/content/depression-detection/processed/train_labels.npz', train_labels)
np.savez('/content/depression-detection/processed/test_samples.npz', test_samples)
np.savez('/content/depression-detection/processed/test_labels.npz', test_labels)

# upload npz files to S3 bucket for accessibility on AWS
print("Uploading npz to S3...")
save_to_bucket('/content/depression-detection/processed/train_samples.npz', 'train_samples.npz')
save_to_bucket('/content/depression-detection/processed/train_labels.npz', 'train_labels.npz')
save_to_bucket('/content/depression-detection/processed/test_samples.npz', 'test_samples.npz')
save_to_bucket('/content/depression-detection/processed/test_labels.npz', 'test_labels.npz')

#### CNN

In [ ]:
"""
CNN used to classify spectrograms of normal participants (0) or depressed
participants (1).
"""


def retrieve_from_bucket(file):
    """
    Download spectrogram representation of matrices from S3 bucket.
    """
    conn = boto.connect_s3(access_key, access_secret_key)
    bucket = conn.get_bucket('depression-detect')
    file_key = bucket.get_key(file)
    file_key.get_contents_to_filename(file)
    X = np.load(file)
    return X


def preprocess(X_train, X_test):
    """
    Convert from float64 to float32 and normalize normalize to decibels
    relative to full scale (dBFS) for the 4 sec clip.
    """
    X_train = X_train.astype('float32')
    X_test = X_test.astype('float32')

    X_train = np.array([(X - X.min()) / (X.max() - X.min()) for X in X_train])
    X_test = np.array([(X - X.min()) / (X.max() - X.min()) for X in X_test])
    return X_train, X_test


def prep_train_test(X_train, y_train, X_test, y_test, nb_classes):
    """
    Prep samples ands labels for Keras input by noramalzing and converting
    labels to a categorical representation.
    """
    print('Train on {} samples, validate on {}'.format(X_train.shape[0],
                                                       X_test.shape[0]))

    # normalize to dBfS
    X_train, X_test = preprocess(X_train, X_test)

    # Convert class vectors to binary class matrices
    Y_train = np_utils.to_categorical(y_train, nb_classes)
    Y_test = np_utils.to_categorical(y_test, nb_classes)

    return X_train, X_test, Y_train, Y_test


def keras_img_prep(X_train, X_test, img_dep, img_rows, img_cols):
    """
    Reshape feature matrices for Keras' expexcted input dimensions.
    For 'th' (Theano) dim_order, the model expects dimensions:
    (# channels, # images, # rows, # cols).
    """
    if K.image_dim_ordering() == 'th':
        X_train = X_train.reshape(X_train.shape[0], 1, img_rows, img_cols)
        X_test = X_test.reshape(X_test.shape[0], 1, img_rows, img_cols)
        input_shape = (1, img_rows, img_cols)
    else:
        X_train = X_train.reshape(X_train.shape[0], img_rows, img_cols, 1)
        X_test = X_test.reshape(X_test.shape[0], img_rows, img_cols, 1)
        input_shape = (img_rows, img_cols, 1)
    return X_train, X_test, input_shape


def cnn(X_train, y_train, X_test, y_test, batch_size,
        nb_classes, epochs, input_shape):
    """
    The Convolutional Neural Net architecture for classifying the audio clips
    as normal (0) or depressed (1).
    """
    model = Sequential()

    model.add(Conv2D(32, (3, 3), padding='valid', strides=1,
                     input_shape=input_shape, activation='relu'))

    model.add(MaxPooling2D(pool_size=(4, 3), strides=(1, 3)))

    model.add(Conv2D(32, (1, 3), padding='valid', strides=1,
              input_shape=input_shape, activation='relu'))

    model.add(MaxPooling2D(pool_size=(1, 3), strides=(1, 3)))

    model.add(Flatten())
    model.add(Dense(512, activation='relu'))
    model.add(Dense(512, activation='relu'))
    model.add(Dropout(0.5))

    model.add(Dense(nb_classes))
    model.add(Activation('softmax'))

    model.compile(loss='categorical_crossentropy',
                  optimizer='adadelta',
                  metrics=['accuracy'])

    history = model.fit(X_train, y_train, batch_size=batch_size, epochs=epochs,
                        verbose=1, validation_data=(X_test, y_test))

    # Evaluate accuracy on test and train sets
    score_train = model.evaluate(X_train, y_train, verbose=0)
    print('Train accuracy:', score_train[1])
    score_test = model.evaluate(X_test, y_test, verbose=0)
    print('Test accuracy:', score_test[1])

    return model, history


def model_performance(model, X_train, X_test, y_train, y_test):
    """
    Evaluation metrics for network performance.
    """
    y_test_pred = model.predict_classes(X_test)
    y_train_pred = model.predict_classes(X_train)

    y_test_pred_proba = model.predict_proba(X_test)
    y_train_pred_proba = model.predict_proba(X_train)

    # Converting y_test back to 1-D array for confusion matrix computation
    y_test_1d = y_test[:, 1]

    # Computing confusion matrix for test dataset
    conf_matrix = standard_confusion_matrix(y_test_1d, y_test_pred)
    print("Confusion Matrix:")
    print(conf_matrix)

    return y_train_pred, y_test_pred, y_train_pred_proba, \
        y_test_pred_proba, conf_matrix


def standard_confusion_matrix(y_test, y_test_pred):
    """
    Make confusion matrix with format:
                  -----------
                  | TP | FP |
                  -----------
                  | FN | TN |
                  -----------
    """
    [[tn, fp], [fn, tp]] = confusion_matrix(y_test, y_test_pred)
    return np.array([[tp, fp], [fn, tn]])


def save_to_bucket(file, obj_name):
    """
    Saves local file to S3 bucket for redundancy and repreoducibility
    by others.
    """
    conn = boto.connect_s3(access_key, access_secret_key)

    bucket = conn.get_bucket('depression-detect')

    file_object = bucket.new_key(obj_name)
    file_object.set_contents_from_filename(file)

In [ ]:
model_id = input("Enter model id: ")

# Load from S3 bucket
print('Retrieving from S3...')
X_train = retrieve_from_bucket('train_samples.npz')
y_train = retrieve_from_bucket('train_labels.npz')
X_test = retrieve_from_bucket('test_samples.npz')
y_test = retrieve_from_bucket('test_labels.npz')

X_train, y_train, X_test, y_test = \
    X_train['arr_0'], y_train['arr_0'], X_test['arr_0'], y_test['arr_0']

# CNN parameters
batch_size = 32
nb_classes = 2
epochs = 7

# normalalize data and prep for Keras
print('Processing images for Keras...')
X_train, X_test, y_train, y_test = prep_train_test(X_train, y_train,
                                                    X_test, y_test,
                                                    nb_classes=nb_classes)

# 513x125x1 for spectrogram with crop size of 125 pixels
img_rows, img_cols, img_depth = X_train.shape[1], X_train.shape[2], 1

# reshape image input for Keras
# used Theano dim_ordering (th), (# chans, # images, # rows, # cols)
X_train, X_test, input_shape = keras_img_prep(X_train, X_test, img_depth,
                                              img_rows, img_cols)

# run CNN
print('Fitting model...')
model, history = cnn(X_train, y_train, X_test, y_test, batch_size,
                      nb_classes, epochs, input_shape)

# evaluate model
print('Evaluating model...')
y_train_pred, y_test_pred, y_train_pred_proba, y_test_pred_proba, \
    conf_matrix = model_performance(model, X_train, X_test, y_train, y_test)

# save model to locally
print('Saving model locally...')
model_name = '../models/cnn_{}.h5'.format(model_id)
model.save(model_name)

# custom evaluation metrics
print('Calculating additional test metrics...')
accuracy = float(conf_matrix[0][0] + conf_matrix[1][1]) / np.sum(conf_matrix)
precision = float(conf_matrix[0][0]) / (conf_matrix[0][0] + conf_matrix[0][1])
recall = float(conf_matrix[0][0]) / (conf_matrix[0][0] + conf_matrix[1][0])
f1_score = 2 * (precision * recall) / (precision + recall)
print("Accuracy: {}".format(accuracy))
print("Precision: {}".format(precision))
print("Recall: {}".format(recall))
print("F1-Score: {}".format(f1_score))

# plot train/test loss and accuracy. saves files in working dir
print('Saving plots...')
plot_loss(history, model_id)
plot_accuracy(history, model_id)
plot_roc_curve(y_test[:, 1], y_test_pred_proba[:, 1], model_id)

# save model S3
print('Saving model to S3...')
save_to_bucket(model_name, 'cnn_{}.h5'.format(model_id))

#### Plot Evaluation Metrics

In [ ]:
def plot_accuracy(history, model_id):
    """
    Plots train and test accuracy for each epoch.
    """
    plt.plot(history.history['acc'])
    plt.plot(history.history['val_acc'])
    plt.title('model accuracy')
    plt.ylabel('accuracy')
    plt.xlabel('epoch')
    plt.legend(['train', 'test'], loc='upper left')
    plt.savefig('images/cnn{}_accuracy.png'.format(model_id))
    plt.close()


def plot_loss(history, model_id):
    """
    Plots train and test loss for each epoch.
    """
    plt.plot(history.history['loss'])
    plt.plot(history.history['val_loss'])
    plt.title('model loss')
    plt.ylabel('loss')
    plt.xlabel('epoch')
    plt.legend(['train', 'test'], loc='upper left')
    plt.savefig('images/cnn{}_loss.png'.format(model_id))
    plt.close()


def plot_roc_curve(y_test, y_score, model_id):
    """
    Plots ROC curve for final trained model. Code taken from:
    https://vkolachalama.blogspot.com/2016/05/keras-implementation-of-mlp-neural.html
    """
    fpr, tpr, _ = roc_curve(y_test, y_score)
    roc_auc = auc(fpr, tpr)
    plt.figure()
    plt.plot(fpr, tpr, label='ROC curve (area = %0.2f)' % roc_auc)
    plt.plot([0, 1], [0, 1], 'k--')
    plt.xlim([0.0, 1.05])
    plt.ylim([0.0, 1.05])
    plt.xlabel('False Positive Rate')
    plt.ylabel('True Positive Rate')
    plt.title('Receiver operating characteristic curve')
    plt.legend(loc="lower right")
    plt.savefig('images/cnn{}_roc.png'.format(model_id))
    plt.close()